# Week 5: Document Similarity

In some applications, it may be difficult to define the classes that we want to use in classification ahead of time.  Or, classes might be made up various subclasses (which differ in terms of the vocabulary used).  In both of these cases (and others), it might be more appropriate to think about **document similarity**.  For a new document, can we find the most similar document in our collection?

### Preliminaries

In [ ]:
###uncomment if working on colab

#from google.colab import drive
#drive.mount('/content/drive')


In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Now lets get a document collection.  We are going to use the Gutenberg collection of books.  We will get the tokenised content of each book and store it in a dictionary (key = the fileid of the book) for easy access.

In [3]:
# from nltk.corpus import gutenberg
import nltk
nltk.download('gutenberg')

book_ids=gutenberg.fileids()
books={b:gutenberg.words(b) for b in book_ids}

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [4]:
books[book_ids[0]]

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', ...]

We now need to normalise the tokens in the documents and construct a *bag-of-words* document representation.  Combining some of the functionality we have been working over the past few weeks (which we have imported from utils.py), we could use something like this

In [9]:
from nltk.corpus import stopwords
from nltk.probability import FreqDist

stop = stopwords.words('english')

def normalise(wordlist):
    lowered=[word.lower() for word in wordlist]
    filtered=[word for word in lowered if word.isalpha() and word not in stop]
    return filtered

book_reps={key:FreqDist(normalise(book)) for key,book in books.items()}

Let's have a look at the representation of first book:

In [15]:
# print(book_reps[book_ids[0]].items())

## Measuring Similarity
We are going to use the cosine measure to determine how similar two books are.  This can be defined in terms of the dot products of vectors:

\begin{eqnarray*}
\mbox{sim}_{\mbox{cosine}}(A,B) = \frac{A.B}{\sqrt{A.A \times B.B}}
\end{eqnarray*}

where the dot product of two vectors, A and B, is defined as:

\begin{eqnarray*}
A.B = \sum_{\mbox{f}} \mbox{weight}(A,f)\times \mbox{weight}(B,f) 
\end{eqnarray*}

and $\mbox{weight}(X,f)$ tells us the value associated with feature $f$ in the vector representation of $X$

### Notes

- One of the reasons that Cosine Similarity is so populat and useful is because it is scale/magnitue invariant. This means that the similarity is based only on the angle, not the length
- A longer document will naturually have higher counts which will lead to a large vector length (magnitude)
- Thematically related documents will show up as different due to lengths, not content
- Cosine similarity divides the dot product by the product of the magnitudes (L2 Normalization), effectively normalizing the vectors to a unit length. This makes the score independent of the document size.
- Measures like Euclidean distance will begin to struggle as high dimensions and struggle from Curse of Dimensionality where the distances between all points tend to become large and approximately equal, making the metric less discriminatory.
- Cosine Similarity is easily interpretable:
    - **1 = perfect allignment, vectors point in same direction;**
    - **0 = orthogonal, no relationship;**
    - **-1 = Perfect opposition (vectors point in diametrically opposite directions)**

### Exercise 1.1
* Write a function `dot` which takes two documents (represented as dictionaries or `FreqDist`s) and returns their dot product
* Test it out on the first two books in Gutenberg.  You should get the answer 3882298!
* Why is the number so large?

In [21]:
# Get the first two books
book1_id = book_ids[0]  # austen-emma.txt
book2_id = book_ids[1]  # austen-persuasion.txt

doc1 = book_reps[book1_id]
doc2 = book_reps[book2_id]

def dot(doc_a, doc_b):
    """
    Calculates the dot product of two document frequency representations.
    
    Args:
        doc_a (dict or FreqDist): Frequency distribution of the first document.
        doc_b (dict or FreqDist): Frequency distribution of the second document.
        
    Returns:
        int: The dot product.
    """
    dot_product = 0
    
    # Set removes 0 counts etc. Minimises set size
    keys_a = set(doc_a.keys())
    keys_b = set(doc_b.keys())
    
    # Intersection where words exist in both
    common_keys = keys_a.intersection(keys_b)
    
    # Calculate the sum of the products of common word frequencies
    for word in common_keys:
        # FreqDist/dict lookup returns the count (frequency)
        dot_product += doc_a[word] * doc_b[word]
        
    return dot_product

# Test the function
result = dot(doc1, doc2)
print(f"Dot product of '{book1_id}' and '{book2_id}': {result}") 
# Result should be 3882298

Dot product of 'austen-emma.txt' and 'austen-persuasion.txt': 3882298


## Notes

- Dictionaries are implicitly representing sparse vectors.
- A sparse vector is a vector in which most of the elements are zero.
- Dense vectors are things link lists and sets.
- You loop only over the common words found via set intersection. This is highly efficient since you skip all the zero-count dimensions.

### Exercise 1.2
* Write a function `cos_sim` which takes two documents (represented as dictionaries or `FreqDist`s) and returns their cosine similarity.
* Your function should make 3 calls to the `dot` function you have already defined
* If you test it out on the first two documents in the finance collection you should get 0.72 (to 2S.F.)

In [34]:
import math
import nltk
from nltk.corpus import reuters
from nltk.probability import FreqDist

nltk.download('reuters')

def dot(doc_a, doc_b):
    """
    Calculates the dot product of two document frequency representations 
    (sparse vectors represented as FreqDists or dictionaries).
    
    Args:
        doc_a (dict or FreqDist): Frequency distribution of the first document.
        doc_b (dict or FreqDist): Frequency distribution of the second document.
        
    Returns:
        int: The dot product.
    """
    dot_product = 0
    
    # Efficiently find the intersection of words present in both documents
    keys_a = set(doc_a.keys())
    keys_b = set(doc_b.keys())
    common_keys = keys_a.intersection(keys_b)
    
    # Calculate the sum of the products of common word frequencies
    for word in common_keys:
        dot_product += doc_a[word] * doc_b[word]
        
    return dot_product

def cos_sim(doc_a, doc_b):
    """
    Calculates the cosine similarity between two documents (FreqDists).
    
    Cosine Similarity = (A . B) / (||A|| * ||B||)
    
    Args:
        doc_a (FreqDist): Frequency distribution of the first document.
        doc_b (FreqDist): Frequency distribution of the second document.
        
    Returns:
        float: The cosine similarity score (between 0 and 1 for positive counts).
    """
    # 1. Calculate the dot product (A . B)
    numerator = dot(doc_a, doc_b)
    
    # Handle the edge case where there is no overlap at all (division by zero is imminent)
    if numerator == 0:
        return 0.0
    
    # 2. Calculate the magnitude of A (||A|| = sqrt(A . A))
    # We use dot(A, A) to calculate the sum of squared frequencies
    mag_a = math.sqrt(dot(doc_a, doc_a))
    
    # 3. Calculate the magnitude of B (||B|| = sqrt(B . B))
    mag_b = math.sqrt(dot(doc_b, doc_b))
    
    # Denominator is ||A|| * ||B||
    denominator = mag_a * mag_b
    
    # Final calculation
    return numerator / denominator

# --- Test Case Setup (Reuters Corpus) ---

# Get the file IDs for the 'finance' category
# finance_files = reuters.fileids('finance')
finance_files = reuters.fileids()

# Select the first two documents
file1_id = finance_files[0]
file2_id = finance_files[1]

# Function to preprocess and create FreqDist
def create_doc_rep(file_id):
    # Get words, convert to lowercase, and filter out punctuation/single characters
    words = [w.lower() for w in reuters.words(file_id) if w.isalnum() and len(w) > 1]
    return FreqDist(words)

# Create the document representations (FreqDists)
doc1_rep = create_doc_rep(file1_id)
doc2_rep = create_doc_rep(file2_id)

print(f"--- Testing Cosine Similarity ---")
print(f"Document 1: {file1_id}")
print(f"Document 2: {file2_id}")

# Calculate and print the result
similarity_score = cos_sim(doc1_rep, doc2_rep)

print("-" * 35)
print(f"Dot Product (A . B): {dot(doc1_rep, doc2_rep)}")
print(f"Magnitude ||A||: {math.sqrt(dot(doc1_rep, doc1_rep)):.2f}")
print(f"Magnitude ||B||: {math.sqrt(dot(doc2_rep, doc2_rep)):.2f}")
print("-" * 35)
print(f"Cosine Similarity: {similarity_score:.4f}")
print(f"Target Check (0.72): {similarity_score:.2f}")

--- Testing Cosine Similarity ---
Document 1: test/14826
Document 2: test/14828
-----------------------------------
Dot Product (A . B): 635
Magnitude ||A||: 72.77
Magnitude ||B||: 15.46
-----------------------------------
Cosine Similarity: 0.5644
Target Check (0.72): 0.56


[nltk_data] Downloading package reuters to
[nltk_data]     /Users/lukebirkett/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


## Notes

- The reason that $\sqrt{\mathbf{A} \cdot \mathbf{A}}$ gives you the magnitude (or length) of vector $\mathbf{A}$ is due to the Pythagorean theorem and the definition of the dot product.
- In basic geometry, the length of a vector $\mathbf{A}$ in 2D space ($\mathbf{A} = [A_1, A_2]$) is found using the Pythagorean theorem: $\text{Length} = \sqrt{A_1^2 + A_2^2}$
- This extends to any dimensions: $\text{Magnitude } ||\mathbf{A}|| = \sqrt{\sum_{i=1}^{n} A_i^2}$
- Dot product is a shortcut to squared: $\mathbf{A} \cdot \mathbf{A} = \sum_{i=1}^{n} A_i \times A_i$
- To find the magnitude ($||\mathbf{A}||$) itself, you just take the square root of both sides: $||\mathbf{A}|| = \sqrt{\mathbf{A} \cdot \mathbf{A}}$

### Exercise 1.3
* Write some code that will compute the similarity of every document in a collection with every document in another collection
* Write code to compute the average similarity of two collections
* Compute (and display) the average similarity of the book collection to itself
    

In [45]:
# read in corpus
# create vectors of file_ids
# place into matrix
# create a functions to calculate the dot products and cos sim
# compute

### Two doc similarity matrix

In [67]:
import math
import nltk
from nltk.corpus import reuters, gutenberg
from nltk.probability import FreqDist
import sys
from collections import defaultdict

# --- NLTK Corpus Download Checks ---
# (Omitted repeated download logic for brevity, assumed to be run previously)

# --- Similarity Functions ---

def dot(doc_a, doc_b):
    """Calculates the dot product of two document frequency representations."""
    dot_product = 0
    keys_a = set(doc_a.keys())
    keys_b = set(doc_b.keys())
    common_keys = keys_a.intersection(keys_b)
    
    for word in common_keys:
        # Use .get(word, 0) for robustness, though FreqDist handles missing keys gracefully
        dot_product += doc_a.get(word, 0) * doc_b.get(word, 0)
        
    return dot_product

def cos_sim(doc_a, doc_b):
    """Calculates the cosine similarity between two documents (FreqDists)."""
    numerator = dot(doc_a, doc_b)
    
    if numerator == 0:
        return 0.0
    
    # Calculate magnitudes: ||A|| = sqrt(A . A)
    mag_a = math.sqrt(dot(doc_a, doc_a))
    mag_b = math.sqrt(dot(doc_b, doc_b))
    
    denominator = mag_a * mag_b
    
    if denominator == 0:
        return 0.0

    return numerator / denominator

# --- Document Preprocessing Function ---

def create_doc_rep(corpus_source, file_id):
    """Gets words from the specified corpus, preprocesses them, and creates a FreqDist."""
    corpus = gutenberg if corpus_source == 'gutenberg' else reuters
    words = [w.lower() for w in corpus.words(file_id) if w.isalnum() and len(w) > 1] # TODO: Remove stop words
    return FreqDist(words)

# --- Collection Setup (Assuming downloads are complete) ---

LIMIT = 5 # SAMPLE OF COLLECTIONS

# Collection 1: Gutenberg Books
gutenberg_files = gutenberg.fileids()[:LIMIT]
gutenberg_collection = {
    fid: create_doc_rep('gutenberg', fid) for fid in gutenberg_files # proccesses > freq dist > dictionary {filename:Bag-of-Words}
}

finance_files = reuters.fileids()[:LIMIT]
reuters_collection = {
    fid: create_doc_rep('reuters', fid) for fid in finance_files
}

# --- Exercise 1.3, Part 1: Compute All Pairwise Similarities ---

print("\n--- Exercise 1.3, Part 1: Pairwise Similarity Matrix ---")
print(f"Comparing {len(gutenberg_collection)} Gutenberg documents (Rows) against {len(reuters_collection)} Reuters documents (Columns).\n")

# Store results in a dictionary where key is the pair (Gutenberg ID, Reuters ID)
similarity_matrix = {}
gutenberg_ids = list(gutenberg_collection.keys())
reuters_ids = list(reuters_collection.keys())

# Outer loop: Iterate through Collection A (Gutenberg)
for g_id in gutenberg_ids:
    g_doc = gutenberg_collection[g_id]
    
    # Inner loop: Iterate through Collection B (Reuters)
    for r_id in reuters_ids:
        r_doc = reuters_collection[r_id]
        
        similarity = cos_sim(g_doc, r_doc)
        similarity_matrix[(g_id, r_id)] = similarity 
        
        # NOTE THIS IS A CONCEPTUAL MATRIX IN DICT FORM
        # REFERENCED USING similarity_matrix[('austen-sense.txt', 'test/14826')]
        # YOU CAN IMAGINE THAT THE FILE NAMES ARE THE ROW AND COL AT ANY INTERSECT
        # IF YOU WANTED TO LOOP THROUGH THE DICT YOU WOULD UNPACK THE KEYS AND COMBINE
        # for (g_id, r_id), similarity in similarity_matrix.items():
        # ACCESS MIN MAX USING
        # most_similar_pair, max_score = max(similarity_matrix.items(), key=lambda item: item[1])

similarity_matrix


--- Exercise 1.3, Part 1: Pairwise Similarity Matrix ---
Comparing 5 Gutenberg documents (Rows) against 5 Reuters documents (Columns).



{('austen-emma.txt', 'test/14826'): 0.6990508219606616,
 ('austen-emma.txt', 'test/14828'): 0.5210845796118742,
 ('austen-emma.txt', 'test/14829'): 0.5270836145598962,
 ('austen-emma.txt', 'test/14832'): 0.3566578418230858,
 ('austen-emma.txt', 'test/14833'): 0.4716770234364965,
 ('austen-persuasion.txt', 'test/14826'): 0.7352379490021591,
 ('austen-persuasion.txt', 'test/14828'): 0.5495933524288018,
 ('austen-persuasion.txt', 'test/14829'): 0.5740870233893807,
 ('austen-persuasion.txt', 'test/14832'): 0.37840912635574064,
 ('austen-persuasion.txt', 'test/14833'): 0.4806626042634651,
 ('austen-sense.txt', 'test/14826'): 0.7192064821343386,
 ('austen-sense.txt', 'test/14828'): 0.5277445411458346,
 ('austen-sense.txt', 'test/14829'): 0.5510520810848243,
 ('austen-sense.txt', 'test/14832'): 0.37137623980338824,
 ('austen-sense.txt', 'test/14833'): 0.4879439180416038,
 ('bible-kjv.txt', 'test/14826'): 0.7345602743193906,
 ('bible-kjv.txt', 'test/14828'): 0.600705201547581,
 ('bible-kjv.txt

In [74]:
all_similarities = similarity_matrix.values()
total_sum = sum(all_similarities)
count = len(all_similarities)

average_similarity = total_sum / count

print(f"Average Similarity (Python): {average_similarity:.6f}")

Average Similarity (Python): 0.532387


In [80]:
LIMIT = 5 # SAMPLE OF COLLECTIONS

# Collection 1: Gutenberg Books
gutenberg_files = gutenberg.fileids()[:LIMIT]
gutenberg_collection = {
    fid: create_doc_rep('gutenberg', fid) for fid in gutenberg_files # proccesses > freq dist > dictionary {filename:Bag-of-Words}
}

similarity_matrix = {}
gutenberg_ids = list(gutenberg_collection.keys())

for g_id_1 in gutenberg_ids:
    g_doc_1 = gutenberg_collection[g_id_1]
    
    for g_id_2 in gutenberg_ids:
        g_doc_2 = gutenberg_collection[g_id_2]
        
        similarity = cos_sim(g_doc_1, g_doc_2)
        similarity_matrix[(g_id_1, g_id_2)] = similarity 
    
similarity_matrix

{('austen-emma.txt', 'austen-emma.txt'): 1.0,
 ('austen-emma.txt', 'austen-persuasion.txt'): 0.972601491890471,
 ('austen-emma.txt', 'austen-sense.txt'): 0.9745607724805279,
 ('austen-emma.txt', 'bible-kjv.txt'): 0.7883510086647902,
 ('austen-emma.txt', 'blake-poems.txt'): 0.7731805641072257,
 ('austen-persuasion.txt', 'austen-emma.txt'): 0.972601491890471,
 ('austen-persuasion.txt', 'austen-persuasion.txt'): 1.0,
 ('austen-persuasion.txt', 'austen-sense.txt'): 0.9735350105647143,
 ('austen-persuasion.txt', 'bible-kjv.txt'): 0.8334059915176326,
 ('austen-persuasion.txt', 'blake-poems.txt'): 0.8148398416369143,
 ('austen-sense.txt', 'austen-emma.txt'): 0.9745607724805279,
 ('austen-sense.txt', 'austen-persuasion.txt'): 0.9735350105647143,
 ('austen-sense.txt', 'austen-sense.txt'): 0.9999999999999999,
 ('austen-sense.txt', 'bible-kjv.txt'): 0.7956832606338139,
 ('austen-sense.txt', 'blake-poems.txt'): 0.7821380724609821,
 ('bible-kjv.txt', 'austen-emma.txt'): 0.7883510086647902,
 ('bible

## Notes

- That is a profoundly important observation, and it highlights the key conceptual difference between sparse vector models (like Bag-of-Words) and dense vector models (like Word2Vec or BERT) in NLP!
- Baseline only considered the intersectioned words
- The measure only tells you how much the two documents overlap relative to their own content, not relative to a universal, fixed, conceptual space.
- Your intuition points to the true limitation of this method: it captures lexical similarity (do they use the same words?) but completely misses semantic similarity (do they use different words to talk about the same idea?).
- The cosine similarity will be low because it sees large vs. big and dog vs. hound as completely different, even though they mean the same thing.
- This limitation is the primary motivation for the shift from sparse models to modern dense embedding models (like Word2Vec, GloVe, or BERT), where words are mapped to continuous, dense vectors that capture meaning, not just frequency.
- However, for your exercise, the sparse dictionary method is the correct and expected implementation for calculating cosine similarity on simple Bag-of-Words document representations.

## Turning a Conceptual Matrix (Dict, Sparse) into a normal matrix (vectorized, dense)

- moving from a sparse data representation (the dictionary) to one optimized for numerical computation and indexing (a NumPy array or Pandas DataFrame)
- To convert your dictionary-based conceptual matrix into a dense, index-based matrix, you need to first establish the definitive order of your rows and columns.
- The most efficient way to achieve this is using the Pandas library, which is built on top of NumPy and excels at handling labeled data.

### Dict -> Series (Pandas) -> Dataframe (Pandas) -> Numpy

In [63]:
import pandas as pd
import numpy as np
# Assuming the 'similarity_matrix' dictionary from your previous code is available here.
# For demonstration purposes, we'll create a small sample dictionary:

similarity_matrix = {
    ('austen-sense.txt', 'test/14826'): 0.534,
    ('chesterton-ball.txt', 'test/14826'): 0.311,
    ('austen-sense.txt', 'test/14842'): 0.701,
    ('chesterton-ball.txt', 'test/14842'): 0.450,
    ('austen-sense.txt', 'test/14845'): 0.622,
    ('chesterton-ball.txt', 'test/14845'): 0.389
}

# --- Conversion to a Vectorized Matrix (Pandas DataFrame) ---

# 1. Create a Series from the dictionary.
# The keys (tuples) automatically become a MultiIndex (a hierarchical index).
similarity_series = pd.Series(similarity_matrix)

# 2. Convert the Series into a DataFrame using the .unstack() method.
# The unstack() operation moves the inner level of the MultiIndex (Reuters ID)
# to become the column headings, creating the final matrix structure.
vectorized_matrix = similarity_series.unstack()

print("--- Vectorized Similarity Matrix (Pandas DataFrame) ---")
print(vectorized_matrix)
print("\n--- Accessing by Row/Column Name ---")
print(f"Similarity (austen-sense.txt vs test/14842): {vectorized_matrix.loc['austen-sense.txt', 'test/14842']:.4f}")

# --- Conversion to a Pure NumPy Array (for Index-Only Access) ---

# If you only need the raw numbers without the row/column names:
numpy_matrix = vectorized_matrix.values # values is pandas property/attribute turning df into numpy

print("\n--- Pure NumPy Array ---")
print(numpy_matrix)
print("\n--- Accessing by Row/Column Index ---")
# Accessing Row 0 (austen-sense.txt), Column 2 (test/14845)
print(f"Similarity (Index [0, 2]): {numpy_matrix[0, 2]:.4f}")

--- Vectorized Similarity Matrix (Pandas DataFrame) ---
                     test/14826  test/14842  test/14845
austen-sense.txt          0.534       0.701       0.622
chesterton-ball.txt       0.311       0.450       0.389

--- Accessing by Row/Column Name ---
Similarity (austen-sense.txt vs test/14842): 0.7010

--- Pure NumPy Array ---
[[0.534 0.701 0.622]
 [0.311 0.45  0.389]]

--- Accessing by Row/Column Index ---
Similarity (Index [0, 2]): 0.6220


### Dict -> Numpy Directly

In [66]:
import numpy as np

# --- 1. Example Similarity Matrix (Using the provided data) ---
# NOTE: In your full solution, you would use the actual dictionary created
# by your nested loops.
similarity_matrix = {
    ('austen-sense.txt', 'test/14826'): 0.534,
    ('chesterton-ball.txt', 'test/14826'): 0.311,
    ('austen-sense.txt', 'test/14842'): 0.701,
    ('chesterton-ball.txt', 'test/14842'): 0.450,
    ('austen-sense.txt', 'test/14845'): 0.622,
    ('chesterton-ball.txt', 'test/14845'): 0.389
}

print("--- Dictionary to NumPy Matrix (Manual Mapping) ---")

# --- 2. Extract and Map Keys ---

# Get all unique row keys (Gutenberg IDs) and sort them
row_keys = sorted(list(set(k[0] for k in similarity_matrix.keys())))

# Get all unique column keys (Reuters IDs) and sort them
col_keys = sorted(list(set(k[1] for k in similarity_matrix.keys())))

# Create mapping dictionaries: String Key -> Integer Index
row_map = {key: i for i, key in enumerate(row_keys)}
col_map = {key: i for i, key in enumerate(col_keys)}

# --- 3. Initialize NumPy Array ---

rows = len(row_keys)
cols = len(col_keys)

# Create a NumPy array of the correct size, initialized to zeros
numpy_matrix = np.zeros((rows, cols))

# --- 4. Populate the Array ---

# Loop through the similarity dictionary
for (r_key, c_key), sim_score in similarity_matrix.items():
    # Look up the integer indices using our maps
    r_idx = row_map[r_key] # get the id from the map using the filename key from the original matrix
    c_idx = col_map[c_key]
    
    # Assign the similarity score to the correct (row, column) position
    numpy_matrix[r_idx, c_idx] = sim_score

# --- Display Results ---

print(f"\nRow Mapping (Key -> Index): {row_map}")
print(f"Col Mapping (Key -> Index): {col_map}")

print("\nResulting NumPy Matrix:")
print(numpy_matrix)
print("\n--- Accessing by Index ---")
# Example: Row 0 (austen-sense.txt), Col 2 (test/14845)
print(f"Similarity (Index [0, 2]): {numpy_matrix[0, 2]:.4f}")

# You would also keep the row_keys and col_keys lists if you needed
# to reference the names later:
print(f"Row 0 is: {row_keys[0]}")
print(f"Col 2 is: {col_keys[2]}")

--- Dictionary to NumPy Matrix (Manual Mapping) ---

Row Mapping (Key -> Index): {'austen-sense.txt': 0, 'chesterton-ball.txt': 1}
Col Mapping (Key -> Index): {'test/14826': 0, 'test/14842': 1, 'test/14845': 2}

Resulting NumPy Matrix:
[[0.534 0.701 0.622]
 [0.311 0.45  0.389]]

--- Accessing by Index ---
Similarity (Index [0, 2]): 0.6220
Row 0 is: austen-sense.txt
Col 2 is: test/14845


## Beyond Frequency
Frequency of a word in a document does not make a very good weight because some words occur very frequently in all documents.  If two rare words occur in both of our pair of documents, that should add more to their perceived similarity than if two common words occur in both of our pair of documents.

### TF-IDF
A commonly used weight is tf-idf which stands for **term frequency, inverse document frequency**

\begin{eqnarray*}
\mbox{tf-idf}(D_i,f) = tf(D_i,f) \times idf(D_i,f)
\end{eqnarray*}

where $tf(D_i,f)$ is simply the frequency of feature f in document $D_i$
and

\begin{eqnarray*}
idf(D_i,f) = log \frac{N}{df(f)}
\end{eqnarray*}

where $N$ is the total number of documents and $\mbox{df}(f)$ is the number of documents containing $f$:  

\begin{eqnarray*}
df(f)=|\{i|\mbox{freq}(D_i,f)>0\}|
\end{eqnarray*}

The code below will take a list of documents (represented as dictionaries) and compute the document frequency for each feature.  Test it out on one the collection of books.

In [ ]:
def doc_freq(doclist):
    df={}
    for doc in doclist:
        for feat in doc.keys():
            df[feat]=df.get(feat,0)+1
            
    return df
    

In [ ]:
doc_freq(book_reps.values())

### Exercise 2.1
* Write a function which will compute the idf values for features given a list of documents
* Use it to compute idf values for features given the entire list of books in the book collection
    

### Exercise 2.2
* Write a function `convert_to_tfidf` that takes two arguments:
    * a dictionary of documents mapping fileids to documents
        * where each document is represented as a dictionary or FreqDist {feat:freq})
    * a dictionary containing idf values
* and outputs a dictionary of documents where each document is represented as a dictionary or FreqDist with tfidf weights {feat:tfidf}

### Exercise 2.3
* Recompute the average similarity between the collection of books (as in Ex 1.3).
* What do you notice?

### Exercise 2.4
For each book in the collection, find it's most similar book (NOT INCLUDING ITSELF!).
Output your results in a table